In [2]:
import pandas as pd
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.fft import fft, ifft, fftfreq
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt


### Load and create data for FFT and LDA

In [5]:
#Made some edits!
# Load data
leakage_data = pd.read_csv('../../data/Positive_Meters_Data/Aquatrax_SmallLeak.csv')
nonleakage_data = pd.read_csv('../../data/Negative_Meters_Data/Aquatrax_Nonleakage_Data_20accounts.csv')

# Cleaning datasets
nonleakage_data = nonleakage_data.rename({'Usagedate': 'UsageDate', 'Meternumber': 'MeterNumber'}, axis='columns') #rename columns
leakage_data = leakage_data.rename({'Usagedate': 'UsageDate', 'Meternumber': 'MeterNumber'}, axis='columns') #rename columns
leakage_data = leakage_data.iloc[:,:4]

# Combine the datasets
leakage_data['Class'] = 1  # Indicate leakage
nonleakage_data['Class'] = 0  # Indicate non-leakage



combined_data = pd.concat([leakage_data, nonleakage_data])

# Sort by MeterNumber and UsageDate
combined_data.sort_values(by=['MeterNumber', 'UsageDate'], inplace=True)

# Reset index after sorting
combined_data.reset_index(drop=True, inplace=True)

combined_data


,AccountNumber,UsageDate,Value,MeterNumber,Class
0,0050005-28920,01-01-2023 00:00,2,0050005-10450-1,0
1,0050005-28920,01-01-2023 01:00,0,0050005-10450-1,0
2,0050005-28920,01-01-2023 02:00,0,0050005-10450-1,0
3,0050005-28920,01-01-2023 03:00,0,0050005-10450-1,0
4,0050005-28920,01-01-2023 04:00,0,0050005-10450-1,0
...,...,...,...,...,...
299163,5751740-68085,12-31-2022 19:00,1,5751740-10450-1,1
299164,5751740-68085,12-31-2022 20:00,0,5751740-10450-1,1
299165,5751740-68085,12-31-2022 21:00,1,5751740-10450-1,1
299166,5751740-68085,12-31-2022 22:00,2,5751740-10450-1,1


In [7]:
#truncate each account number to have same length = threshold
LEN_THRES = 7500
df_account_list = [combined_data[combined_data['AccountNumber'] == number].tail(LEN_THRES) for number in combined_data['AccountNumber'].unique()]
for (idx, df_account) in enumerate(df_account_list):
    if (len(df_account) < LEN_THRES):  #remove account if total number of datapoints below threshold
        dropping_meter_number = df_account.loc[:].reset_index()['MeterNumber'][0]
        combined_data = combined_data.drop(combined_data[combined_data['MeterNumber']==dropping_meter_number].index)
        del df_account_list[idx]
[len(e) for e in df_account_list]  #display number of data points for each account

[7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500,
 7500]

In [8]:
print(len(df_account_list[0]))

7500


In [10]:
#need to run the above block before running this one!!

#list to store pivoted account dataframes
df_account_pivoted_list = []

#conduct fourier transfor on each account
for (idx, df_account) in enumerate(df_account_list):
    df_account['FFT_F'] = fftfreq(len(df_account), 1)
    df_account['FFT_Period'] = 1/df_account['FFT_F']
    df_account['FFT'] = fft(df_account['Value'])
    df_account['FFTA'] = np.abs(df_account['FFT'])

    #remove large periods
    df_account = df_account.where(np.abs(df_account['FFT_Period']) <= 30)
    df_account = df_account.dropna()
    df_account_list[idx] = df_account

    #pivot each account dataframe
    df_account_pivoted = df_account.pivot(index="AccountNumber", columns='FFT_Period', values='FFTA')
    df_account_pivoted_list.append(df_account_pivoted)

df_account_pivoted_list[0]  #this shows 0 na's in dataframe


FFT_Period,-30.000000,-29.880478,-29.761905,-29.644269,-29.527559,-29.411765,-29.296875,-29.182879,-29.069767,-28.957529,...,28.957529,29.069767,29.182879,29.296875,29.411765,29.527559,29.644269,29.761905,29.880478,30.000000
AccountNumber,,,,,,,,,,,,,,,,,,,,,
0050005-28920,54.685236,99.025246,188.035079,114.800729,144.750595,23.000038,135.396546,129.04646,76.731557,238.048828,...,238.048828,76.731557,129.04646,135.396546,23.000038,144.750595,114.800729,188.035079,99.025246,54.685236


In [11]:
#combine all pivoted account dataframes
df_accounts_pivoted = pd.concat(df_account_pivoted_list)
df_accounts_pivoted

FFT_Period,-30.000000,-29.880478,-29.761905,-29.644269,-29.527559,-29.411765,-29.296875,-29.182879,-29.069767,-28.957529,...,28.957529,29.069767,29.182879,29.296875,29.411765,29.527559,29.644269,29.761905,29.880478,30.000000
AccountNumber,,,,,,,,,,,,,,,,,,,,,
0050005-28920,54.685236,99.025246,188.035079,114.800729,144.750595,23.000038,135.396546,129.046460,76.731557,238.048828,...,238.048828,76.731557,129.046460,135.396546,23.000038,144.750595,114.800729,188.035079,99.025246,54.685236
0050015-328466,90.814993,138.442607,239.481152,181.858867,178.467594,142.203437,206.763755,195.837020,117.969219,155.332205,...,155.332205,117.969219,195.837020,206.763755,142.203437,178.467594,181.858867,239.481152,138.442607,90.814993
0050017-668118,69.589964,42.628413,111.557355,71.018653,82.817134,40.738750,44.993817,168.380393,57.583345,264.435066,...,264.435066,57.583345,168.380393,44.993817,40.738750,82.817134,71.018653,111.557355,42.628413,69.589964
0050020-660260,70.888698,66.252341,93.928796,147.942500,68.286694,148.980265,52.302249,172.284751,87.091262,57.550918,...,57.550918,87.091262,172.284751,52.302249,148.980265,68.286694,147.942500,93.928796,66.252341,70.888698
0050100-50430,162.693075,342.090265,144.246943,344.418958,187.838806,244.506983,45.115045,214.846123,233.952047,134.429189,...,134.429189,233.952047,214.846123,45.115045,244.506983,187.838806,344.418958,144.246943,342.090265,162.693075
0050110-26,26.262934,74.651877,93.774045,148.685412,32.879654,42.683590,26.424128,136.837330,106.586450,58.831910,...,58.831910,106.586450,136.837330,26.424128,42.683590,32.879654,148.685412,93.774045,74.651877,26.262934
0050180-66814,37.606374,142.358608,62.846760,12.793066,65.517646,78.002774,25.867598,26.464456,13.975070,53.481037,...,53.481037,13.975070,26.464456,25.867598,78.002774,65.517646,12.793066,62.846760,142.358608,37.606374
0050200-530518,86.389453,96.846777,132.291912,122.379976,146.598468,117.817455,77.786461,306.032763,70.663081,235.832302,...,235.832302,70.663081,306.032763,77.786461,117.817455,146.598468,122.379976,132.291912,96.846777,86.389453
0050240-149921,332.748656,60.483246,277.190162,340.697592,524.663173,165.630675,242.616275,746.565831,408.541545,397.397238,...,397.397238,408.541545,746.565831,242.616275,165.630675,524.663173,340.697592,277.190162,60.483246,332.748656


In [16]:
print(len(df_account_pivoted_list))

33


### Perform FFT

In [92]:
# fft_features = []

# for meter_number in combined_data['MeterNumber'].unique():
#     meter_data = combined_data[combined_data['MeterNumber'] == meter_number]
#     fft_result = np.abs(fft(meter_data['Value']))
#     fft_features.append(fft_result)

# # Create a new DataFrame with FFT features
# fft_df = pd.DataFrame(fft_features)

# fft_df.head()


In [93]:
#testing data formats
# np.array(df_accounts_pivoted) #m meter accounts, n fft features, m x n matrix
# combined_data.loc[combined_data['MeterNumber'] == '0050005-10450-1']['Class'][0]
# meter_number = '0050005-10450-1'
# combined_data.loc[combined_data['MeterNumber'] == meter_number]['Class'][0]

# [combined_data.loc[combined_data['MeterNumber'] == meter_number]['Class'][0] for meter_number in combined_data['MeterNumber'].unique()]

### Perform and evaluate LDA

In [17]:
# Perform LDA
X = np.array(df_accounts_pivoted)
y = np.array([combined_data.loc[combined_data['MeterNumber'] == meter_number].reset_index()['Class'][0] for meter_number in combined_data['MeterNumber'].unique()])
lda = LinearDiscriminantAnalysis()
lda.fit(X, y)
lda.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [18]:
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
y_pred = lda.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.7857142857142857
Confusion Matrix:
[[10  0]
 [ 3  1]]


In [25]:
lda_coefficients = lda.coef_

for i in range(33):
    print(lda_coefficients[0][i])

3.3919258277061784e-05
5.0490553147394926e-05
3.676492688064241e-05
-1.2955381728032007e-05
2.6292737197954135e-05
5.196357609688063e-06
1.5701352516348978e-05
-3.390158445520743e-05
-5.432945300765562e-06
1.4523896493456801e-05
1.2172313927822287e-05
1.384220396455828e-05
-5.458056974719585e-06
5.085853254954251e-07
1.3561769687516892e-05
9.851750352872373e-06
3.102246172541112e-08
1.395240144517409e-05
2.0836656257866156e-05
3.110222708533585e-05
2.570255929072779e-05
6.8966434398072096e-06
1.6409941514902173e-05
9.217658385115523e-06
1.4443455245054507e-06
-4.902355542886116e-06
2.3359627268441546e-05
1.158241468597075e-06
5.2198929732816455e-05
3.039744361170738e-06
7.89020706009015e-06
-9.717428665543146e-05
-5.21272973717371e-06


1
